In [11]:
%cd video_part

[WinError 2] The system cannot find the file specified: 'video_part'
c:\Users\alapa\imageupscaler\video_part


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [12]:
import cv2
import os

def video_to_frames(input_video_path, output_frames_folder):
    # Create the output directory if it doesn't exist
    os.makedirs(output_frames_folder, exist_ok=True)
    
    # Open the video file
    video_capture = cv2.VideoCapture(input_video_path)
    success, frame = video_capture.read()
    count = 1
    
    # Iterate over frames and save each one
    while success:
        frame_path = os.path.join(output_frames_folder, f"frame_{count}.jpg")
        cv2.imwrite(frame_path, frame)
        count += 1
        success, frame = video_capture.read()
    
    video_capture.release()
    print(f"Frames saved to {output_frames_folder}")

# Example usage
input_video_path = "input/input_vid.mp4"  # Replace with your video file path
output_frames_folder = "temp"  # Temporary folder for frames
video_to_frames(input_video_path, output_frames_folder)


Frames saved to temp


In [14]:
import cv2
import os

def frames_to_video_and_clear(input_frames_folder, output_video_path, fps=30):
    # Get all frame file names and sort them in order
    frame_files = sorted([f for f in os.listdir(input_frames_folder) if f.endswith(".jpg")],
                         key=lambda x: int(x.split('_')[1].split('.')[0]))
    
    # Read the first frame to determine the video size
    first_frame_path = os.path.join(input_frames_folder, frame_files[0])
    first_frame = cv2.imread(first_frame_path)
    height, width, layers = first_frame.shape
    
    # Initialize the video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Add each frame to the video and delete it from the folder
    for frame_file in frame_files:
        frame_path = os.path.join(input_frames_folder, frame_file)
        frame = cv2.imread(frame_path)
        video_writer.write(frame)
        
        # Delete the frame after adding it to the video
        os.remove(frame_path)
    
    video_writer.release()
    print(f"Video saved to {output_video_path} and temporary frames cleared.")

# Example usage
input_frames_folder = "temp"  # Folder containing frames
output_video_path = "output/reconstructed1_video.mp4"  # Path to save the output video
frames_to_video_and_clear(input_frames_folder, output_video_path)


Video saved to output/reconstructed1_video.mp4 and temporary frames cleared.


### live video procesing


In [16]:
import cv2
import os

def process_live_feed(output_frames_folder, reconstructed_video_path, fps=30):
    # Set up directories
    os.makedirs(output_frames_folder, exist_ok=True)
    frame_count = 0
    
    # Set up video capture for live feed (0 for the default camera)
    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    # Get the width and height of the frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up video writer for real-time reconstructed video
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(reconstructed_video_path, fourcc, fps, (width, height))
    
    print("Processing live feed... Press 'q' to quit.")
    
    # Real-time frame processing loop
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Save each frame temporarily
        frame_count += 1
        frame_path = os.path.join(output_frames_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        # Write each frame to the output video
        video_writer.write(frame)
        
        # Remove the frame file immediately after writing to video
        os.remove(frame_path)
        
        # Display the live feed
        cv2.imshow('Live Feed', frame)
        
        # Quit with 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"Reconstructed video saved to {reconstructed_video_path}")

# Example usage
output_frames_folder = "temp"  # Temporary folder for frames
reconstructed_video_path = "output/reconstructed_video.mp4"  # Save path for the output videoq
process_live_feed(output_frames_folder, reconstructed_video_path)


Processing live feed... Press 'q' to quit.
Reconstructed video saved to output/reconstructed_video.mp4
